# Geocoding of Outbreak Cases

### Requirements for the Excel File that contains the outbreak data:
- Make sure that the sheet contains the headinga "Street", "ZIP" and "City" written exactly like this in the first row of the file
- Make sure the Sheet where the data is stored is called "Sheet1".
- Make sure that the file is stored at the same location as this script and is named "Artificial_Outbreak.xlsx". If that is not the case you have to set the variable "input_file" to the according location.

### What you need to customize in this script:
- Set the Workspace of the ArcGIS project (Options > Current Setting > Default geodatabase)
- change the username to your username for the ArcGIS platform. After executing the following code snippet you will be asked to enter your password. It is necessary to sign in before executing the code thereafter because geocoding locations consumes credits. 

#### [Markdown Guide](https://www.markdownguide.org/basic-syntax/)

In [ ]:
import os
import pathlib
import arcpy
from arcgis.gis import GIS
from getpass import getpass
import pandas as pd

In [ ]:
workspace = 'C:/Users/srude/Documents/ArcGIS/Projects/Spatial Food Consumption Model Montenegro/Spatial Food Consumption Model Montenegro.gdb'
password = getpass()
gis = GIS("http://www.arcgis.com", 'Sandra_Rudeloff', password)

In [ ]:
dirname = pathlib.Path().resolve()
input_file = os.path.join(dirname, 'Artificial_Outbreak.xlsx')

In [ ]:
data = pd.read_excel(input_file, sheet_name='Sheet1')
data.head(10)

In [ ]:
address_locator = "https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/ArcGIS World Geocoding Service"
address_fields  = ("Address Street VISIBLE NONE; Address2 <None> VISIBLE NONE; Address3 <None> VISIBLE NONE;" +
                "Neighborhood <None> VISIBLE NONE; City City VISIBLE NONE; Subregion <None> VISIBLE NONE;" 
                + "Region <None> VISIBLE NONE; ZIP ZIP ZIP VISIBLE NONE; ZIP4 <None> VISIBLE NONE;" +
                "Country <None> VISIBLE NONE")

**TODO**: Hier könnte man noch prüfen, wie gut das geocoding funktioniert hat

In [ ]:
outbreak = workspace + "/artificial_outbreak2"

arcpy.GeocodeAddresses_geocoding(in_table = data, address_locator = address_locator,in_address_fields = address_fields, out_feature_class= outbreak,
                                 out_relationship_type = 'STATIC', country = 'MNE', location_type = 'ROUTING LOCATION', output_fields = 'MINIMAL')

# Algorithm to detect the most susceptible source

 Set the array of patterns to compare (names of the layers); in this case supermarkets

In [ ]:
supermarkets = ["HDL", "Franca", "Aroma", "Voli", "Idea"]
df_distances = pd.DataFrame(columns = ["Supermarket_Chain", "Outbreak Case", "Chain Store", "Distance"])

**TODO**: Check what is the difference between Geodesic and Planar 

**PLANAR**—Uses planar distances between the features. This is the default. </br>
**GEODESIC** —Uses geodesic distances between features. This method takes into account the curvature of the spheroid and correctly deals with data near the dateline and poles

[Generate Near Table](https://pro.arcgis.com/en/pro-app/latest/tool-reference/analysis/generate-near-table.htm)

In [40]:
arcpy.env.overwriteOutput = True

def calculate_closest():
    index = 1 
    
    # calculate the distance of the outbreak case to the nearest supermarket of each chain defined in the array supermarkets
    for supermarket in supermarkets:
        arcpy.GenerateNearTable_analysis(outbreak, (workspace+'/'+ supermarket), r'memory\tempOutput', 
        location= "NO_LOCATION", angle= "NO_ANGLE", closest="CLOSEST", method= "Geodesic"
        )

       # write result in dataframe
        with arcpy.da.SearchCursor(r'memory\tempOutput', ["IN_FID","NEAR_FID", "NEAR_DIST"]) as cursor:
            for row in cursor:
                df_distances.loc[index] =[supermarket, row[0], row[1], row[2]]
                index = index + 1
    arcpy.Delete_management(r'memory\tempOutput')
    print(df_distances)

calculate_closest()

   Supermarket_Chain Outbreak Case Chain Store     Distance
1                HDL             1           3  1793.477053
2                HDL             2           1  1416.929901
3                HDL             3           3  1081.899089
4                HDL             4           4  2437.292858
5                HDL             5           3  1407.057989
6                HDL             6           4  2142.854335
7                HDL             7           3  1730.225316
8                HDL             8          11   987.172242
9                HDL             9           4  1921.958580
10               HDL            10           1  1970.253284
11            Franca             1          26   476.882091
12            Franca             2          29   517.667448
13            Franca             3          27   424.970236
14            Franca             4          27   979.516616
15            Franca             5          29  1588.970025
16            Franca             6      

In [41]:
df_new = df_distances.groupby('Supermarket_Chain').sum()
print (df_new)

                       Distance
Supermarket_Chain              
Aroma               5051.573911
Franca              9450.218803
HDL                16889.120647
Idea                4252.976677
Voli                6881.627644


Put a logic that excludes if there is a 0 Value